Importing libraries

In [49]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

Scraping wikipedia page for Toronto table

In [50]:
wiki_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = requests.get(wiki_link).text
soup = BeautifulSoup(req, 'lxml')

In [51]:
table = soup.find('table')
print(table)

Postcode = []
Borough = []
Neighbourhood = []

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [52]:
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)


In [53]:
unique_p = set(Postcode)

Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

In [54]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.head(14)

import geocoder
from geopy.geocoders import Nominatim

In [55]:
nom = Nominatim(user_agent = None, timeout = 20)
df_toronto['Canada']= 'Canada'
df_toronto.head(10)

C:\Users\Camelia Oancea\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


,Borough,Neighbourhood,Postcode,Canada
0,West Toronto,Dovercourt Village,M6H,Canada
1,Central Toronto,"The Annex, Yorkville",M5R,Canada
2,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Sou...",M5V,Canada
3,Etobicoke,The Kingsway,M8X,Canada
4,North York,Hillcrest Village,M2H,Canada
5,Central Toronto,"Deer Park, Rathnelly, South Hill",M4V,Canada
6,Etobicoke,Kingsview Village,M9R,Canada
7,East York,"Woodbine Gardens, Parkview Hill",M4B,Canada
8,West Toronto,"Runnymede, Swansea",M6S,Canada
9,East York,Thorncliffe Park,M4H,Canada


In [56]:
df_toronto['Full Address']=df_toronto['Neighbourhood']+', '+df_toronto['Canada']
df_toronto.head(5)
df_toronto['Coordinates'] = df_toronto['Full Address'].apply(nom.geocode)
df_toronto['Latitude']=df_toronto['Coordinates'].apply(lambda x: x.latitude if x!=None else None)
df_toronto['Longitude'] = df_toronto['Coordinates'].apply(lambda x: x.longitude if x!=None else None)

In [57]:
df_toronto.head(10)

,Borough,Neighbourhood,Postcode,Canada,Full Address,Coordinates,Latitude,Longitude
0,West Toronto,Dovercourt Village,M6H,Canada,"Dovercourt Village, Canada","(Dovercourt, Edmonton, Alberta, Canada, (53.57...",53.575954,-113.559646
1,Central Toronto,"The Annex, Yorkville",M5R,Canada,"The Annex, Yorkville, Canada","(TD Canada Trust, 165, Avenue Road, Bloor Stre...",43.674601,-79.396304
2,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Sou...",M5V,Canada,"CN Tower, King and Spadina, Railway Lands, Sou...",None,NaN,NaN
3,Etobicoke,The Kingsway,M8X,Canada,"The Kingsway, Canada","(The Kingsway, Etobicoke, Toronto, Ontario, M8...",43.647381,-79.511333
4,North York,Hillcrest Village,M2H,Canada,"Hillcrest Village, Canada","(Hillcrest Village, North York, Toronto, Ontar...",43.799664,-79.365019
5,Central Toronto,"Deer Park, Rathnelly, South Hill",M4V,Canada,"Deer Park, Rathnelly, South Hill, Canada",None,NaN,NaN
6,Etobicoke,Kingsview Village,M9R,Canada,"Kingsview Village, Canada","(Kingsview Village, Etobicoke, Toronto, Ontari...",43.699539,-79.556346
7,East York,"Woodbine Gardens, Parkview Hill",M4B,Canada,"Woodbine Gardens, Parkview Hill, Canada",None,NaN,NaN
8,West Toronto,"Runnymede, Swansea",M6S,Canada,"Runnymede, Swansea, Canada","(Runnymede, 1, Runnymede Road, Swansea, Old To...",43.651703,-79.475998
9,East York,Thorncliffe Park,M4H,Canada,"Thorncliffe Park, Canada","(Thorncliffe Park, East York, Toronto, Ontario...",43.704553,-79.345407


In [58]:
df_toronto2 = df_toronto["Neighbourhood"].str.split(",", n = 1, expand = True)
df_toronto['First Address']=df_toronto2[0]
df_toronto['Final Address'] = df_toronto['First Address'] + ', '+df_toronto['Canada']
df_toronto['Coordinates2'] = df_toronto['Final Address'].apply(nom.geocode)
df_toronto['Lat']=df_toronto['Coordinates2'].apply(lambda x: x.latitude if x!=None else None)
df_toronto['Long'] = df_toronto['Coordinates2'].apply(lambda x: x.longitude if x!=None else None)

In [59]:
print(df_toronto2.isnull().sum())

0     0
1    48
dtype: int64


In [60]:
df_toronto.head(10)

,Borough,Neighbourhood,Postcode,Canada,Full Address,Coordinates,Latitude,Longitude,First Address,Final Address,Coordinates2,Lat,Long
0,West Toronto,Dovercourt Village,M6H,Canada,"Dovercourt Village, Canada","(Dovercourt, Edmonton, Alberta, Canada, (53.57...",53.575954,-113.559646,Dovercourt Village,"Dovercourt Village, Canada","(Dovercourt, Edmonton, Alberta, Canada, (53.57...",53.575954,-113.559646
1,Central Toronto,"The Annex, Yorkville",M5R,Canada,"The Annex, Yorkville, Canada","(TD Canada Trust, 165, Avenue Road, Bloor Stre...",43.674601,-79.396304,The Annex,"The Annex, Canada","(The Annex, Old Toronto, Toronto, Ontario, M2R...",43.675790,-79.402560
2,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Sou...",M5V,Canada,"CN Tower, King and Spadina, Railway Lands, Sou...",None,NaN,NaN,CN Tower,"CN Tower, Canada","(CN Tower, 301, Front Street West, Entertainme...",43.642564,-79.387087
3,Etobicoke,The Kingsway,M8X,Canada,"The Kingsway, Canada","(The Kingsway, Etobicoke, Toronto, Ontario, M8...",43.647381,-79.511333,The Kingsway,"The Kingsway, Canada","(The Kingsway, Etobicoke, Toronto, Ontario, M8...",43.647381,-79.511333
4,North York,Hillcrest Village,M2H,Canada,"Hillcrest Village, Canada","(Hillcrest Village, North York, Toronto, Ontar...",43.799664,-79.365019,Hillcrest Village,"Hillcrest Village, Canada","(Hillcrest Village, North York, Toronto, Ontar...",43.799664,-79.365019
5,Central Toronto,"Deer Park, Rathnelly, South Hill",M4V,Canada,"Deer Park, Rathnelly, South Hill, Canada",None,NaN,NaN,Deer Park,"Deer Park, Canada","(Deer Park, Old Toronto, Toronto, Ontario, M4V...",43.688090,-79.394093
6,Etobicoke,Kingsview Village,M9R,Canada,"Kingsview Village, Canada","(Kingsview Village, Etobicoke, Toronto, Ontari...",43.699539,-79.556346,Kingsview Village,"Kingsview Village, Canada","(Kingsview Village, Etobicoke, Toronto, Ontari...",43.699539,-79.556346
7,East York,"Woodbine Gardens, Parkview Hill",M4B,Canada,"Woodbine Gardens, Parkview Hill, Canada",None,NaN,NaN,Woodbine Gardens,"Woodbine Gardens, Canada","(Woodbine Gardens, East York, Toronto, Ontario...",43.712078,-79.302567
8,West Toronto,"Runnymede, Swansea",M6S,Canada,"Runnymede, Swansea, Canada","(Runnymede, 1, Runnymede Road, Swansea, Old To...",43.651703,-79.475998,Runnymede,"Runnymede, Canada","(Runnymede, Old Toronto, Toronto, Ontario, M6P...",43.665580,-79.482108
9,East York,Thorncliffe Park,M4H,Canada,"Thorncliffe Park, Canada","(Thorncliffe Park, East York, Toronto, Ontario...",43.704553,-79.345407,Thorncliffe Park,"Thorncliffe Park, Canada","(Thorncliffe Park, East York, Toronto, Ontario...",43.704553,-79.345407


In [61]:
df_toronto.isna().sum()

Borough           0
Neighbourhood     0
Postcode          0
Canada            0
Full Address      0
Coordinates      22
Latitude         22
Longitude        22
First Address     0
Final Address     0
Coordinates2      2
Lat               2
Long              2
dtype: int64

In [62]:
print(df_toronto)

             Borough                                      Neighbourhood  \
0       West Toronto                                 Dovercourt Village   
1    Central Toronto                               The Annex, Yorkville   
2   Downtown Toronto  CN Tower, King and Spadina, Railway Lands, Sou...   
3          Etobicoke                                       The Kingsway   
4         North York                                  Hillcrest Village   
5    Central Toronto                   Deer Park, Rathnelly, South Hill   
6          Etobicoke                                  Kingsview Village   
7          East York                    Woodbine Gardens, Parkview Hill   
8       West Toronto                                 Runnymede, Swansea   
9          East York                                   Thorncliffe Park   
10        North York                                    Maple Leaf Park   
11   Central Toronto                                         Moore Park   
12         Etobicoke     

In [63]:
df_toronto.iloc[29:54, :]

,Borough,Neighbourhood,Postcode,Canada,Full Address,Coordinates,Latitude,Longitude,First Address,Final Address,Coordinates2,Lat,Long
29,Downtown Toronto,Berczy Park,M5E,Canada,"Berczy Park, Canada","(Berczy Park, 35, King East, Old Toronto, Toro...",43.648001,-79.375385,Berczy Park,"Berczy Park, Canada","(Berczy Park, 35, King East, Old Toronto, Toro...",43.648001,-79.375385
30,Etobicoke,"Beaumond Heights, Jamestown, Mount Olive, Silv...",M9V,Canada,"Beaumond Heights, Jamestown, Mount Olive, Silv...",None,NaN,NaN,Beaumond Heights,"Beaumond Heights, Canada",None,NaN,NaN
31,Downtown Toronto,Ryerson,M5B,Canada,"Ryerson, Canada","(Ryerson, Census Division No. 1, Saskatchewan,...",49.784979,-101.668968,Ryerson,"Ryerson, Canada","(Ryerson, Census Division No. 1, Saskatchewan,...",49.784979,-101.668968
32,Downtown Toronto,"Cabbagetown, St. James Town",M4X,Canada,"Cabbagetown, St. James Town, Canada",(British-Israel World Federation (Canada) Inc....,43.661517,-79.372140,Cabbagetown,"Cabbagetown, Canada","(Cabbagetown, Old Toronto, Toronto, Ontario, M...",43.664473,-79.366986
33,York,Weston,M9N,Canada,"Weston, Canada","(Weston, York, Toronto, Ontario, M9N 2B3, Cana...",43.700161,-79.516247,Weston,"Weston, Canada","(Weston, York, Toronto, Ontario, M9N 2B3, Cana...",43.700161,-79.516247
34,Downtown Toronto,"First Canadian Place, Underground city",M5X,Canada,"First Canadian Place, Underground city, Canada",None,NaN,NaN,First Canadian Place,"First Canadian Place, Canada","(First Canadian Place, 100, King Street West, ...",43.648768,-79.381692
35,North York,Flemingdon Park,M3C,Canada,"Flemingdon Park, Canada","(Flemingdon Park, North York, Toronto, Ontario...",43.718432,-79.333204,Flemingdon Park,"Flemingdon Park, Canada","(Flemingdon Park, North York, Toronto, Ontario...",43.718432,-79.333204
36,Central Toronto,Lawrence Park,M4N,Canada,"Lawrence Park, Canada","(Lawrence Park, Old Toronto, Toronto, Ontario,...",43.729199,-79.403252,Lawrence Park,"Lawrence Park, Canada","(Lawrence Park, Old Toronto, Toronto, Ontario,...",43.729199,-79.403252
37,North York,Bedford Park,M5M,Canada,"Bedford Park, Canada","(Bedford Park, North York, Toronto, Ontario, M...",43.737388,-79.410925,Bedford Park,"Bedford Park, Canada","(Bedford Park, North York, Toronto, Ontario, M...",43.737388,-79.410925
38,Downtown Toronto,Church and Wellesley,M4Y,Canada,"Church and Wellesley, Canada","(Wellesley, 1, Yonge Street, Church-Wellesley ...",43.665524,-79.383801,Church and Wellesley,"Church and Wellesley, Canada","(Wellesley, 1, Yonge Street, Church-Wellesley ...",43.665524,-79.383801


In [64]:
df_toronto.iloc[30,11] = nom.geocode('Thistletown, Canada').latitude

In [65]:
df_toronto.iloc[30]

Borough                                                  Etobicoke
Neighbourhood    Beaumond Heights, Jamestown, Mount Olive, Silv...
Postcode                                                       M9V
Canada                                                      Canada
Full Address     Beaumond Heights, Jamestown, Mount Olive, Silv...
Coordinates                                                   None
Latitude                                                       NaN
Longitude                                                      NaN
First Address                                     Beaumond Heights
Final Address                             Beaumond Heights, Canada
Coordinates2                                                  None
Lat                                                        43.7373
Long                                                           NaN
Name: 30, dtype: object

In [66]:
df_toronto.iloc[30,12] = nom.geocode('Thistletown, Canada').longitude

In [67]:
df_toronto.iloc[30,:]

Borough                                                  Etobicoke
Neighbourhood    Beaumond Heights, Jamestown, Mount Olive, Silv...
Postcode                                                       M9V
Canada                                                      Canada
Full Address     Beaumond Heights, Jamestown, Mount Olive, Silv...
Coordinates                                                   None
Latitude                                                       NaN
Longitude                                                      NaN
First Address                                     Beaumond Heights
Final Address                             Beaumond Heights, Canada
Coordinates2                                                  None
Lat                                                        43.7373
Long                                                      -79.5653
Name: 30, dtype: object

In [68]:
df_toronto.iloc[60:62,:]

,Borough,Neighbourhood,Postcode,Canada,Full Address,Coordinates,Latitude,Longitude,First Address,Final Address,Coordinates2,Lat,Long
60,North York,"Newtonbrook, Willowdale",M2M,Canada,"Newtonbrook, Willowdale, Canada","(Newtonbrook Boulevard, Newton Brook, North Yo...",43.783882,-79.423253,Newtonbrook,"Newtonbrook, Canada","(Newtonbrook Secondary School, 155, Hilda Aven...",43.793886,-79.425679
61,North York,CFB Toronto,M3K,Canada,"CFB Toronto, Canada",None,NaN,NaN,CFB Toronto,"CFB Toronto, Canada",None,NaN,NaN


In [69]:
df_toronto.iloc[61,11] = nom.geocode('Canadian Forces Base, Toronto, Canada').latitude

In [70]:
df_toronto.iloc[61,:]

Borough                   North York
Neighbourhood            CFB Toronto
Postcode                         M3K
Canada                        Canada
Full Address     CFB Toronto, Canada
Coordinates                     None
Latitude                         NaN
Longitude                        NaN
First Address            CFB Toronto
Final Address    CFB Toronto, Canada
Coordinates2                    None
Lat                          43.7516
Long                             NaN
Name: 61, dtype: object

In [71]:
df_toronto.iloc[61, 12] = nom.geocode('Canadian Forces Base, Toronto, Canada').longitude

In [72]:
df_toronto.iloc[61,:]

Borough                   North York
Neighbourhood            CFB Toronto
Postcode                         M3K
Canada                        Canada
Full Address     CFB Toronto, Canada
Coordinates                     None
Latitude                         NaN
Longitude                        NaN
First Address            CFB Toronto
Final Address    CFB Toronto, Canada
Coordinates2                    None
Lat                          43.7516
Long                        -79.4712
Name: 61, dtype: object

In [73]:
df_toronto.isna().sum()

Borough           0
Neighbourhood     0
Postcode          0
Canada            0
Full Address      0
Coordinates      22
Latitude         22
Longitude        22
First Address     0
Final Address     0
Coordinates2      2
Lat               0
Long              0
dtype: int64

In [74]:
df_toronto.columns

Index(['Borough', 'Neighbourhood', 'Postcode', 'Canada', 'Full Address',
       'Coordinates', 'Latitude', 'Longitude', 'First Address',
       'Final Address', 'Coordinates2', 'Lat', 'Long'],
      dtype='object')

In [75]:
df_toronto_fin = df_toronto.iloc[:,[2,0,1,11,12]]

In [76]:
df_toronto_fin.head(10)

,Postcode,Borough,Neighbourhood,Lat,Long
0,M6H,West Toronto,Dovercourt Village,53.575954,-113.559646
1,M5R,Central Toronto,"The Annex, Yorkville",43.675790,-79.402560
2,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Sou...",43.642564,-79.387087
3,M8X,Etobicoke,The Kingsway,43.647381,-79.511333
4,M2H,North York,Hillcrest Village,43.799664,-79.365019
5,M4V,Central Toronto,"Deer Park, Rathnelly, South Hill",43.688090,-79.394093
6,M9R,Etobicoke,Kingsview Village,43.699539,-79.556346
7,M4B,East York,"Woodbine Gardens, Parkview Hill",43.712078,-79.302567
8,M6S,West Toronto,"Runnymede, Swansea",43.665580,-79.482108
9,M4H,East York,Thorncliffe Park,43.704553,-79.345407


In [77]:
df_toronto_fin = df_toronto_fin.rename(index = str, columns = {'Lat':'Latitude', 'Long': 'Longitude'})

In [78]:
df_toronto_fin.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M6H,West Toronto,Dovercourt Village,53.575954,-113.559646
1,M5R,Central Toronto,"The Annex, Yorkville",43.675790,-79.402560
2,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Sou...",43.642564,-79.387087
3,M8X,Etobicoke,The Kingsway,43.647381,-79.511333
4,M2H,North York,Hillcrest Village,43.799664,-79.365019
5,M4V,Central Toronto,"Deer Park, Rathnelly, South Hill",43.688090,-79.394093
6,M9R,Etobicoke,Kingsview Village,43.699539,-79.556346
7,M4B,East York,"Woodbine Gardens, Parkview Hill",43.712078,-79.302567
8,M6S,West Toronto,"Runnymede, Swansea",43.665580,-79.482108
9,M4H,East York,Thorncliffe Park,43.704553,-79.345407
